In [ ]:
import os
import math

import numpy as np
import fasttext.util

import nn
import utils


embeddings = utils.get_embeddings(["en", "fr"])

en_emb = embeddings["en"]
fr_emb = embeddings["fr"]

In [ ]:
class TranslationDataset():
    def __init__(self, inputs, targets, embeddings):
        self.inputs = inputs 
        self.targets = targets 
        
        # Encoders for both languages
        en_i = embeddings["en"]
        en_t = embeddings["fr"]
        
        self.input_em = []
        for seq in inputs:
            self.input_em.append(np.array([en_i.get_word_vector(w) for w in seq.split()]))
            
        self.target_em = []
        for seq in targets:
            self.target_em.append(np.array([en_t.get_word_vector(w) for w in seq]))

    def __len__(self):
        return (len(self.sequence))

    def __getitem__(self, idx):
        return({
            "input":self.inputs[idx],
            "target":self.targets[idx],
            "input_embedding":self.input_em[idx],
            "target_embedding":self.target_em[idx],
        })
    
data = ["the cat likes oranges", "hello my friend"]
target = ["le chat aime les oranges", "bonjour mon amie"]

dataset = TranslationDataset(data, target, embeddings) 

print(dataset[0])

In [ ]:
def pos_encoding(seq):
    """Adds positional encoding to a sequence of word vectors"""
    seq_len = seq.shape[0]
    d_model = seq.shape[1]
    
    encoding = []
    for i in range(seq_len):
        w = 1 / (10000 ** ((2 * i) / d_model))

        wi_s = [math.sin(p * w) * (i % 2) for p in range(d_model)]
        wi_c = [math.cos(p * w) * ((i + 1) % 2) for p in range(d_model)]
        
        encoding.append(np.add(wi_s, wi_c))
        
    encoding = np.array(encoding)
    
    return np.add(encoding, seq)
    

print(dataset[0]["input"])

pos_encoding(dataset[0]["input_embedding"])

In [122]:
class AttentionHead():
    """Scaled dot product attention head. """
    def __init__(self, embed_dim):
        L = nn.LinearLayer(embed_dim, embed_dim)
    
    def forward(self, Q, V, K):
        """Attention forward pass"""
        d_k = K.shape[1]
        scale = math.sqrt(d_k)
        
        return(nn.softmax(np.dot(Q, K.T) / scale), V)
        
        
class MultiheadAttention():
    """Multiheaded attention transformer block"""
    def __init__(self):
        pass
        
X = dataset[0]["input_embedding"]

head = AttentionHead(X)
head.forward(X, X, X)

TypeError: __init__() missing 1 required positional argument: 'embed_dim'